# GPT-dev

Notebook for developing first steps into building GPT from scratch.

### Imports

In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

### Data Loading

Downloading Tiny Sheakespeare Dataset for a light version training of GPT:

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2023-10-17 03:58:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  --.-KB/s    in 0,1s    

2023-10-17 03:58:55 (8,39 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
if not os.path.exists("../data"):
    os.makedirs("../data")

In [4]:
!mv input.txt ../data/

### Data Inspection

In [5]:
with open("../data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

Datset length in characters:

In [6]:
len(text)

1115394

First 200 characters:

In [7]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


Unique characters:

In [8]:
chars = sorted(list(set(text)))
len(chars)

65

In [9]:
"".join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

### Text Tokenization

Translate characters to integers in a simple manner:
> OpenAI uses [tiktoken](https://github.com/openai/tiktoken)

In [10]:
str_to_int_dict = {ch:i for i,ch in enumerate(chars)}
int_to_str_dict = {i:ch for i,ch in enumerate(chars)}

def encode(string):
    return [str_to_int_dict[char] for char in string]

def decode(list_of_int):
    return "".join([int_to_str_dict[i] for i in list_of_int])

In [11]:
encode("hello world")

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]

In [12]:
decode([46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42])

'hello world'

In [13]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


First 100 encoded characters:

In [14]:
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

### Data Split

Train and test sets (90/10%):

In [15]:
n = int(0.9*len(data))

train_data = data[:n]
test_data = data[n:]

### Data Loader

Generate batches of chunks of data:

In [16]:
#Example
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [17]:
torch.manual_seed(77)

batch_size = 4
block_size = 8

def get_batch(split: str):
    
    if split == "train":
        data = train_data
    elif split == "val":
        data = val_data
    else:
        return
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [18]:
xb, yb = get_batch("train")

print("Inputs:", xb.shape)
print(xb)

print("Outputs:", yb.shape)
print(yb)

Inputs: torch.Size([4, 8])
tensor([[ 1, 51, 39, 56, 56, 47, 43, 42],
        [33, 24, 21, 17, 32, 10,  0, 21],
        [ 0,  0, 15, 24, 13, 30, 17, 26],
        [42,  1, 39, 52, 42,  1, 57, 43]])
Outputs: torch.Size([4, 8])
tensor([[51, 39, 56, 56, 47, 43, 42,  1],
        [24, 21, 17, 32, 10,  0, 21,  1],
        [ 0, 15, 24, 13, 30, 17, 26, 15],
        [ 1, 39, 52, 42,  1, 57, 43, 39]])


Input-target pairs:

In [19]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target: {target}")

When input is [1] the target: 51
When input is [1, 51] the target: 39
When input is [1, 51, 39] the target: 56
When input is [1, 51, 39, 56] the target: 56
When input is [1, 51, 39, 56, 56] the target: 47
When input is [1, 51, 39, 56, 56, 47] the target: 43
When input is [1, 51, 39, 56, 56, 47, 43] the target: 42
When input is [1, 51, 39, 56, 56, 47, 43, 42] the target: 1
When input is [33] the target: 24
When input is [33, 24] the target: 21
When input is [33, 24, 21] the target: 17
When input is [33, 24, 21, 17] the target: 32
When input is [33, 24, 21, 17, 32] the target: 10
When input is [33, 24, 21, 17, 32, 10] the target: 0
When input is [33, 24, 21, 17, 32, 10, 0] the target: 21
When input is [33, 24, 21, 17, 32, 10, 0, 21] the target: 1
When input is [0] the target: 0
When input is [0, 0] the target: 15
When input is [0, 0, 15] the target: 24
When input is [0, 0, 15, 24] the target: 13
When input is [0, 0, 15, 24, 13] the target: 30
When input is [0, 0, 15, 24, 13, 30] the targ

### Baseline
Bigram Language Model

In [20]:
torch.manual_seed(77)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx) #(B,T,C)
        
        if targets is None:
            loss = None
        else:     
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # (B*T,C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            # get predictions
            logits, loss = self(idx)
            
            # focus only on the last time step
            logits = logits[:, -1, :] #(B,C)
            probs = F.softmax(logits, dim=-1)
            
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
        return idx
            

In [21]:
vocab_size = len(chars)
m = BigramLanguageModel(vocab_size)

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.7358, grad_fn=<NllLossBackward0>)


In [22]:
idx = torch.zeros((1,1), dtype=torch.long)
gen_m = m.generate(idx, max_new_tokens=100)[0].tolist()
print(decode(gen_m))


CuRTR;rF&xcn?ttd.o:xnDYUxTlN$?;&gLJ Yf-&UGhd.K,MmdzkFIRrn!DRdqKMgGfrbM3muNxfP&ZCKL?jKZWIfeweQNze3WI;


Training:

In [23]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
n_steps = 1000

for steps in range(n_steps):
    
    # batch of data
    xb, yb = get_batch("train")
    
    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

3.6724908351898193


Generation:

In [24]:
idx = torch.zeros((1,1), dtype=torch.long)
gen_m = m.generate(idx, max_new_tokens=300)[0].tolist()
print(decode(gen_m))


EtVc!&XirBfmYWo:u?fwrD3FZ-oT?:UvilKw3a
A.w-o jjURKielK,bNOL.Tf gNOfcCh oEI;M.JILvT?msosteQE$uaXVy f-Sh-Ed taQNIAm:ggLnJyWuvV:Tj;rt haGonqGyyH rw3NnatRAcliSYWHl$ fakDcYWrNDAPEggv;WucVazQ:xE$&:VJe'nPkFs.L gq&SKY cBbh3ORcBTh .GRuvecZewVQVats.Z-m:ND&QJPJvdz;rQoHlSLV&kDJVmF&'d yE. YWs.LVHE$f btNace?:
BQt


### Self-attention
Mathematical trick:
- Matrix multiplication
- Softmax

In [25]:
# Example
torch.manual_seed(77)

B,T,C = 4,8,2
x = torch.rand(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [26]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev,0)
        
xbow.shape

torch.Size([4, 8, 2])

In [27]:
torch.manual_seed(77)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[5., 5.],
        [8., 6.],
        [7., 7.]])
c=
tensor([[5.0000, 5.0000],
        [6.5000, 5.5000],
        [6.6667, 6.0000]])


In [28]:
weigths = torch.tril(torch.ones(T, T))
weigths = weigths / weigths.sum(dim=1, keepdim=True)
weigths

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Matrix multiplication:

In [29]:
xbow2 = weigths @ x #(B,T,T) @ (B,T,C) --> (B,T,C)
torch.allclose(xbow, xbow2)

True

In [30]:
xbow[0], xbow2[0]

(tensor([[0.2919, 0.2857],
         [0.3470, 0.3751],
         [0.5481, 0.3355],
         [0.5772, 0.4669],
         [0.5325, 0.4404],
         [0.5652, 0.4636],
         [0.5551, 0.5378],
         [0.5499, 0.5028]]),
 tensor([[0.2919, 0.2857],
         [0.3470, 0.3751],
         [0.5481, 0.3355],
         [0.5772, 0.4669],
         [0.5325, 0.4404],
         [0.5652, 0.4636],
         [0.5551, 0.5378],
         [0.5499, 0.5028]]))

Softmax:

In [32]:
tril = torch.tril(torch.ones(T,T))
weigths = torch.zeros((T,T))
weigths = weigths.masked_fill(tril == 0, float("-inf"))
weigths = F.softmax(weigths, dim=-1)

xbow3 = weigths @ x
torch.allclose(xbow, xbow3)

True

In [33]:
a = torch.rand((4,8,2)) 
b = torch.ones((8,2))
print(a)
print(b)
print(a+b)

tensor([[[0.6645, 0.8609],
         [0.3538, 0.3347],
         [0.7285, 0.5794],
         [0.4946, 0.9831],
         [0.5134, 0.2580],
         [0.5002, 0.6152],
         [0.6283, 0.6972],
         [0.5420, 0.5651]],

        [[0.0891, 0.8486],
         [0.1901, 0.3035],
         [0.6512, 0.4057],
         [0.8156, 0.6753],
         [0.9405, 0.6498],
         [0.6329, 0.6110],
         [0.1847, 0.5707],
         [0.6729, 0.3982]],

        [[0.2390, 0.2084],
         [0.7921, 0.1592],
         [0.9983, 0.4600],
         [0.6405, 0.8427],
         [0.8737, 0.6508],
         [0.1006, 0.5123],
         [0.2961, 0.0813],
         [0.3255, 0.7400]],

        [[0.5685, 0.9510],
         [0.2281, 0.7452],
         [0.9047, 0.1822],
         [0.9578, 0.7193],
         [0.4830, 0.3722],
         [0.2317, 0.9074],
         [0.6607, 0.4544],
         [0.9629, 0.2350]]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
  